## Import Libraries and Load Data

In [21]:
# Import libraries
import warnings

import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy

# Set seed
SEED = 4031
np.random.seed(SEED)

In [22]:
# Data file locations and names

project_root_dir = "Data"
project_subdir_prefix = "fold_"
train_data_filename = "train.csv"
test_data_filename = "test.csv"


# The number of train/test data folders and the target RMSE for each
# train/test split in each folder

n_datasets = 10

In [23]:
# Get list of data subfolders, each with a separate training and test set.

os_walk = os.walk(project_root_dir)
data_subdir_list = [subdirs for root, subdirs, files in os_walk][0]
n_subdirs = len(data_subdir_list)

assert(n_subdirs == n_datasets)

In [24]:
# Lists for training and test datasets

train_datasets = []
test_datasets = []


# Loop over subfolders and read in training/test datasets and test weekly sales.
# Use a loop instead of using os.walk directly to avoid "fold10" immediately following "fold1".

for subdir_num in np.arange(n_subdirs) + 1:
    subdir_num_str = str(subdir_num)
    train_datasets.append(pd.read_csv(os.path.join(project_root_dir,
                                                   project_subdir_prefix + subdir_num_str,
                                                   train_data_filename)))
    test_datasets.append(pd.read_csv(os.path.join(project_root_dir,
                                                   project_subdir_prefix + subdir_num_str,
                                                   test_data_filename)))

## Define Scoring function

In [25]:
def myeval():
    file_path = 'Data/test_with_label.csv'
    test_with_label = pd.read_csv(file_path)
    num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

## Clean negative sales

In [26]:
#for dataset in train_datasets:
#    dataset.loc[dataset['Weekly_Sales'] < 0, 'Weekly_Sales'] = 0

## Edit original OLS: group stores by department and add SVD/PCA

Preprocessing steps
1. Group training data by department
2. Pivot each department's data so that stores are rows and dates are columns, with values = weekly sales
3. Fill in missing stores and dates, setting their sales to zero
4. Center store values
5. Perform SVD
6. Re-add store means
7. Use the SVD output as y_train for x_train = \[Year, Week, Store\]

In [27]:
# Components to return from SVD. This is from the example in Campuswire post #364:
# https://campuswire.com/c/G06C55090/feed/364
n_components = 8

In [28]:
def smooth_weekly_sales(train_data, fold, n_components=n_components):
    """
    Given a fold's training dataset of weekly sales by store and department,
    use SVD to smooth each department's weekly sales across stores.
    Return the dataset with smoothed sales.
    """
    
    # Store each department's processed data
    #dept_store_list = []
    smoothed_sales_list = []
    
    t_split = dict(tuple(train_data.groupby(["Dept"])))
    
    depts = list(t_split)
    
    for dept in depts:
        
        # Get needed columns for department
        t_data = t_split[dept]
        
        # Pivot each store/dept combo's sales by date.
        # Rows and stores are depts, columns are dates, values are sales figures.
        t_pivot = t_data.pivot(index=["Store", "Dept"], columns="Date", values="Weekly_Sales").reset_index().fillna(0)


        # Save list of Date values for dept
        t_dates = t_pivot.columns[2:]

      
        # Extract just the dates and sales for further processing
        t_sales = t_pivot.to_numpy()[:, 2:]
        t_dept_store = t_pivot.to_numpy()[:, :2]
        
        # Get store means
        t_store_means = np.mean(t_sales, axis=1)[:, np.newaxis]


        # Center the sales by store
        t_centered_sales = t_sales - t_store_means
        
        # Perform SVD on centered sales data if there are more than n_components components
        if t_centered_sales.shape[0] > n_components:
            U, S, V = np.linalg.svd(t_centered_sales)
            
            # Keep only the top n_components components
            U_reduced = U[:, :n_components]
            S_reduced = np.diag(S[:n_components])
            V_reduced = V[:n_components, :]
            
            # Reconstruct smoothed sales
            t_sales_smooth = np.dot(U_reduced, np.dot(S_reduced, V_reduced)) + t_store_means
        else:
            # If there are not enough components to perform SVD, use original sales
            t_sales_smooth = t_sales
        
        #print(t_sales_smooth)
        # Join Store and Dept back to smoothed sales figures
        t_dept_store_sales_smooth = np.concatenate((t_dept_store, t_sales_smooth), axis=1)
        t_columns = ["Store", "Dept"] + t_dates.tolist()
        
        #print("t_columns:", t_columns)
    
        # Convert smoothed sales from array to dataframe
        t_dept_store_sales_smooth_df = pd.DataFrame(t_dept_store_sales_smooth, columns=t_columns)

        # Unpivot the smoothed sales
        t_sales_unpivot = t_dept_store_sales_smooth_df.melt(id_vars=["Store", "Dept"], 
                                                            var_name="Date",
                                                            value_name="Weekly_Sales").reset_index(drop=True)

        # Save dept's smoothed sales
        smoothed_sales_list.append(t_sales_unpivot)


    
    # Stack the accumulated smoothed sales
    t_dept_store_sales = pd.concat(smoothed_sales_list, axis=0, ignore_index=True).reset_index(drop=True)
    
    return t_dept_store_sales

In [29]:
def dates_to_years_and_weeks(data):
    """
    Convert sales dates in data to numeric years and categorical weeks.
    """

    tmp = pd.to_datetime(data["Date"])
    data["Wk"] = tmp.dt.isocalendar().week
    data["Yr"] = tmp.dt.year
    data["Wk"] = pd.Categorical(data["Wk"], categories=[i for i in range(1, 53)])  # 52 weeks
    
    return data

In [30]:
# Loop over folds
for j in range(n_datasets):

    # Get a pair of training and test sets
    train = train_datasets[j]
    test = test_datasets[j]

    test_pred = pd.DataFrame()

    # Identify the distinct store/dept pairs shared by the training and test set.
    # Will only process these.

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])
    
    # Join the distinct store/dept pairs to the training set.
    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    
    # Now join the distinct store/dept pairs to the test set.
    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    
    #print('pre-smooth')
    #print(train_split)

    # Smooth weekly sales in the training dataset
    train_smooth = smooth_weekly_sales(train_split, j, n_components)
    
    # Convert training sales dates to years + weeks
    train_split = dates_to_years_and_weeks(train_smooth)
    train_split = train_split.sort_values(by=['Store', 'Dept', 'Date'], ascending=[True, True, True])

    #print('post-smooth')
    #print(train_split)
    
    #print("train_smooth columns:", train_smooth.columns)
    
    # Get design matrices for training y and X.
    # y is just the target variable, Weekly_Sales.
    # X has pivoted weeks, where individual weeks are separate 0/1 columns.
    y, X = patsy.dmatrices('Weekly_Sales ~ Weekly_Sales + Store + Dept + Yr  + Wk', 
                        data = train_split, 
                        return_type='dataframe')

    # Group by store and department to help build separate model for store/dept combo.
    # Create dict where key = (Store, Dept) and value = dataframe of Store/Date/Weekly_Sales.
    #train_split = dict(tuple(train_split.groupby(["Dept"])))
    train_split = dict(tuple(X.groupby(["Store", "Dept"])))

    # Convert test sales dates to years + weeks
    test_split = dates_to_years_and_weeks(test_split)

    # Get design matrices for text y and X.
    # y is the Year, and the design matrix is "Store + Dept + Yr + Wk".
    # Note that test sets don't have the Weekly_Sales target variable.
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk', 
                        data = test_split, 
                        return_type='dataframe')
    
    # Re-add Date column to the design matrix X
    X['Date'] = test_split['Date']
    # Get dictionary where keys are (Store, Dept) tuples, and values are the
    # \"Yr  + Wk + Date\" design matrices corresponding to each key.
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))
    
    # Get the training departments
    keys = list(train_split)

    # Loop over (store, dept) tuples
    for key in keys:
        
        #print("Key:", key)

        # Get training and test design matrices corresponding to (store, dept)
        X_train = train_split[key]
        X_test = test_split[key]
    
        # Target variable for (store, dept)
        Y = X_train['Weekly_Sales']
        # Drop ID and target to get just a table of predictors
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)
        
        # Identify columns that are all zero in training predictors, and drop them
        # from both training and test X.
        # This should drop weeks that are not represented in the training data.
        # How does this affect test X? Are there cases where all test weeks would be dropped?
        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        
        # Identify X training columns that are highly collinear with the columns to the left.
        # Note that this doesn't check the Intercept column.
        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-10:
                    cols_to_drop.append(col_name)
                
        # Drop those collinear columns from both training and test X.
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        #add quadratic Yrs
        #print(X_train.columns)
        try:
            X_train["Yr^2"] = X_train["Yr"]*X_train["Yr"]
            X_test["Yr^2"] = X_test["Yr"]*X_test["Yr"]
        except:
            pass

        # Fit a regular ordinary least squares model on training Weekly_Sales.
        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)
        
        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)
        
        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
        
    test_pred['Weekly_Pred'].fillna(0, inplace=True)

    #Replace negative predictions with 0
    test_pred.loc[test_pred['Weekly_Pred'] < 0, 'Weekly_Pred'] = 0

    #Merge with original test set for this fold to capture unpredicted values (not in training) and set to 0
    test_pred = pd.merge(test_datasets[j], test_pred, how="left", on=["Store", "Dept", "Date"]).fillna(0)
    
    # Save the output to CSV
    file_path = f'Data/fold_{j+1}/mypred.csv'
    print(f'fold_{j+1} processed')
    test_pred.to_csv(file_path, index=False)

KeyboardInterrupt: 

## Evaluate predictions

In [ ]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

	1943.432
	1360.761
	1381.853
	1526.242
	2316.271
	1630.953
	1611.095
	1353.463
	1335.697
	1332.011
1579.178


yr^2 not removing negatives in trainin	

1945.911
1364.181
1384.114
1528.761
2320.364
1638.587
1614.787
1355.643
1337.016
1334.433

1582.380

yr^2 removing negatives in trainin

1946.054
1363.932
1383.931
1528.226
2320.131
1638.557
1614.712
1355.654
1337.036
1334.380

1582.261

yr^3 removing negatives in trainin

1946.051
1363.931
1383.931
1528.226
2320.131
1638.970
1614.709
1355.654
1337.044
1334.387

1582.304

yr^2 removing negatives in training and predictions

1943.624
1360.557
1381.705
1525.914
2316.151
1630.949
1611.057
1353.492
1335.722
1331.992

1579.116

yr^2 only removing negatives in predictions

1943.432
1360.761
1381.853
1526.242
2316.271
1630.953
1611.095
1353.463
1335.697
1332.011

1579.178